In [10]:
# ==============================================================================
# PROJECT: Kaggle Playground Series - S6E2 (Heart Disease)
# MISSION: Advanced Stacking Ensemble (The Meta-Learner)
# NARRATOR: [Your Name]
# ==============================================================================

import pandas as pd
import numpy as np
from google.colab import drive
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# --- STEP 1: MOUNT & LOAD ---
drive.mount('/content/drive')
TRAIN_PATH = '/content/drive/MyDrive/Nihal Data/kaggle/S6E1 - heart/train.csv'
TEST_PATH = '/content/drive/MyDrive/Nihal Data/kaggle/S6E1 - heart/test.csv'

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

# --- STEP 2: PREPARATION ---
target_mapping = {'Absence': 0, 'Presence': 1}
y = train['Heart Disease'].map(target_mapping)
X = train.drop(['id', 'Heart Disease'], axis=1)
X_test = test.drop(['id'], axis=1)

for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category').cat.codes
    X_test[col] = X_test[col].astype('category').cat.codes

# --- STEP 3: DEFINING THE STACK ---

# Level 0 Models: Your optimized XGBoost + a standard LightGBM
base_models = [
    ('xgb', XGBClassifier(
        n_estimators=1000,
        learning_rate=0.0524,
        max_depth=4,
        subsample=0.817,
        colsample_bytree=0.509,
        tree_method='hist',
        device='cuda',
        random_state=42)),

    ('lgb', LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=63,
        device='gpu',
        random_state=42,
        verbosity=-1))
]

# Level 1 Model: The Master Judge
# Logistic Regression decides how to blend the base models' probabilities
meta_learner = LogisticRegression()

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_learner,
    cv=5,
    stack_method='predict_proba',
    n_jobs=-1
)

# --- STEP 4: TRAINING THE STACK ---
print("Training the Stacking Ensemble... This uses Cross-Validation internally.")
stack_model.fit(X, y)

# --- STEP 5: FINAL PREDICTION ---
test_probs = stack_model.predict_proba(X_test)[:, 1]

# --- STEP 6: SUBMISSION ---
submission = pd.DataFrame({'id': test['id'], 'Heart Disease': test_probs})
submission.to_csv('submission_v7_stacked.csv', index=False)

print("Stacked submission 'submission_v7_stacked.csv' generated.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training the Stacking Ensemble... This uses Cross-Validation internally.
Stacked submission 'submission_v7_stacked.csv' generated.
